In [1]:
from scene_edit_utils import *

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
example_idx = "2021.05.12.22.00.38_veh-35_01008_01518"

folder_path = "output/nuplan_example/" + example_idx

resume_from = folder_path + "/checkpoint_final.pth"
new_checkpoint_path = folder_path + "/checkpoint_edit.pth"

trainer_01518 = load_trainer(resume_from)
rigid_01518 = trainer_01518.models['RigidNodes']
print(torch.unique(rigid_01518.point_ids[..., 0]))

/home/haoweis/.conda/envs/drivestudio/lib/python3.9/site-packages/torch/cuda/__init__.py:209: UserWarning: 
NVIDIA H100 80GB HBM3 with CUDA capability sm_90 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_35 sm_50 sm_60 sm_61 sm_70 sm_75 sm_80 sm_86 sm_89 compute_89.
If you want to use the NVIDIA H100 80GB HBM3 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(
Loading lidar:   0%|          | 0/31 [00:00<?, ?it/s]/home/haoweis/drivestudio/datasets/nuplan/nuplan_sourceloader.py:408: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /home/conda/fee

Using predefined pose: da_pose
tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18], device='cuda:0')


In [5]:
print(rigid_01518.instances_trans.shape)

torch.Size([31, 19, 3])


In [ ]:
example_idx = "023"

folder_path = "output/waymo_example/idx_" + example_idx

resume_from = folder_path + "/checkpoint_final.pth"
new_checkpoint_path = folder_path + "/checkpoint_edit.pth"

trainer_023 = load_trainer(resume_from)
rigid_023 = trainer_023.models['RigidNodes']
print(torch.unique(rigid_023.point_ids[..., 0]))

In [ ]:
example_idx = "327"

folder_path = "output/waymo_example/idx_" + example_idx

resume_from = folder_path + "/checkpoint_final.pth"
new_checkpoint_path = folder_path + "/checkpoint_edit.pth"

trainer_327 = load_trainer(resume_from)
rigid_327 = trainer_327.models['RigidNodes']
print(torch.unique(rigid_327.point_ids[..., 0]))

In [ ]:
"""
#rigid_023 = transfer_veh(rigid_023, rigid_327, 1)

new_trans = rigid_023.instances_trans[:, 6].cpu().detach().numpy()

new_trans[:, 0] = np.array([new_trans[0, 0]]*len(new_trans))
#new_trans[:, 1] = np.array([new_trans[0, 1]]*len(new_trans))

rigid_023 = change_trans(rigid_023, 6, torch.tensor(new_trans, device='cuda:0', requires_grad=True).view(len(new_trans), 3))
"""

In [ ]:
rigid_01518 = change_trans_gradual(rigid_01518, 2, 0, 30, [5.2243e+01, -2.7e+00, 3.0053e-03])

In [ ]:
rigid_01518.instances_trans[:, 2]

In [ ]:
"""
example_idx = "2021.05.12.22.00.38_veh-35_01008_01518"

folder_path = "output/nuplan_example/" + example_idx

resume_from = folder_path + "/checkpoint_final.pth"
new_checkpoint_path = folder_path + "/checkpoint_edit.pth"
"""
save_checkpoint(trainer_01518, new_checkpoint_path)

In [ ]:
new_trans = np.array([[10.0, -10.0, 0.0]]*len(rigid_327.instances_trans[:, 0].cpu().detach().numpy()))
rigid_327 = change_trans(rigid_327, 2, torch.tensor(new_trans, device='cuda:0', requires_grad=True).view(len(new_trans), 3))

In [ ]:
background_023 = trainer_023.models["Background"]
background_327 = trainer_327.models["Background"]

In [ ]:
background_327._means # xzy means of gaussian points

In [ ]:
rigid_327.instances_size[0] # xyz size of a vehicle
rigid_327.instances_trans[:, 0] # xyz location of center of bounding box, so lowest point would be z_box - z_size/2

In [ ]:
trainer_327.scene_origin

In [ ]:
test = background_327._means.cpu().detach().numpy()
test_scales = background_327._scales.cpu().detach().numpy()

In [ ]:
sorted_by_first_column = test[np.argsort(test[:, 0])]
sorted_by_second_column = test[np.argsort(test[:, 1])]

# Retrieve entries in the second array where the y values (second column) are between -10.5 and -9.5
filtered_by_y = sorted_by_second_column[(sorted_by_second_column[:, 1] > -10.001) & (sorted_by_second_column[:, 1] < -9.999)]

# Retrieve entries in the first array where the x values (first column) are between 9.5 and 10.5
filtered_by_x = sorted_by_first_column[(sorted_by_first_column[:, 0] > 9.999) & (sorted_by_first_column[:, 0] < 10.001)]

In [ ]:
filtered_by_xy = test[(test[:, 0] > 9.8) & (test[:, 0] < 10.2) & (test[:, 1] > -0.2) & (test[:, 1] < 0.2)]
filtered_by_xy = filtered_by_xy[np.argsort(filtered_by_xy[:, 2])]

# Calculate the Euclidean distance to the point (10, -10) for each entry
distances = np.sqrt((filtered_by_xy[:, 0] - 10)**2 + (filtered_by_xy[:, 1] + 10)**2)

# Find the index of the minimum distance
min_index = np.argmin(distances)

# Select the entry closest to (10, -10)
closest_entry = filtered_by_xy[min_index]

In [ ]:
rigid_327._scales